Наши данные

In [1]:
import pandas as pd
import re

pth = '/home/sondors/Documents/price/BERT_NER/csv/Kazakov_241123/Gislaved_character.xlsx'   
df = pd.read_excel(pth, dtype=str)
df
# print(df.keys())

,PRICE_FRNO,PRICE_FNAM,PRICE_CAID,PRICE_CNAM,PRICE_TMNO,Примечание,Наименование,Модель,Ширина шины,Высота профиля,...,PRICE_PCOD,date,PRICE_RSLT,kpl,ppl,PRICE_IMGN,PRICE_LOCD,lineNumber,PRICE_SALES,PRICE_PLID
0,99353,NaN,8101,NaN,7365,NaN,Gislaved Comspeed 185/75 R16 104/102R,Com Speed,185,75,...,NaN,NaN,Auto,ШИНЫ :: Gislaved,Gislaved,http://pitstop33.ru/img/tire/tumb/1337101116.jpg,"[8101, 2895]",2023-05-26 20:23:08,только 1 шт. Шиномонтаж -25%,1729364159
1,99353,NaN,8101,NaN,7365,NaN,Gislaved Comspeed 185/75 R16C 104/102R,Com Speed,185,75,...,NaN,NaN,Auto,ШИНЫ :: Gislaved,Gislaved,http://pitstop33.ru/img/tire/tumb/1337101116.jpg,"[8101, 2895]",2023-05-26 20:23:08,только 1 шт. Шиномонтаж -25%,1729359059
2,73619,NaN,8101,NaN,7365,NaN,Gislaved Comspeed 185/75 R16C 104R,Com Speed,185,75,...,NaN,NaN,Auto,ШИНЫ :: Gislaved,Gislaved,http://33market.ru/img/tire/tumb/1337101116.jpg,"[8101, 2211]",44945.01600694445,только 1 шт. Шиномонтаж на месте -25%,1657253071
3,61893,NaN,8101,NaN,7365,NaN,Gislaved COM*SPEED 195/60 R16C 99/97T,Com Speed,195,60,...,452193,Active,Auto,Товары для авто :: Летние,Gislaved,https://www.bs-tyres.ru/img/image/Gislaved-COM...,[8101],2023-08-23 08:30:42,Воздух для шин в подарок,1756113660
4,61852,NaN,8101,NaN,0,NaN,185/60 R15 Gislaved Euro Frost 6 88T,Euro Frost 6,185,60,...,3435160000,NaN,Auto,Шины,NaN,NaN,[8101],44911.76752314815,NaN,1647406102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2157,61893,NaN,8101,NaN,7365,NaN,Gislaved Urban Speed 175/65 R13 80T,Urban Speed,175,65,...,1243731,NaN,Auto,Товары для авто :: Летние,Gislaved,https://www.best-tyres.ru/img/image/Gislaved-U...,[8101],45002.7296412037,NaN,1676094924
2158,73619,NaN,8101,NaN,7365,NaN,Gislaved UrbanSpeed 175/65 R13 80T,Urban Speed,175,65,...,NaN,NaN,Auto,ШИНЫ :: Gislaved,Gislaved,http://33market.ru/img/tire/tumb/1332147366.jpg,"[8101, 2499]",44945.01600694445,"Продажа от 1 шт., Шиномонтаж - 25%.",1657256545
2159,73619,NaN,8101,NaN,7365,NaN,Gislaved UrbanSpeed 185/60 R14 82H,Urban Speed,185,60,...,NaN,NaN,Auto,ШИНЫ :: Gislaved,Gislaved,http://33market.ru/img/tire/tumb/1332147366.jpg,"[8101, 2604, 2603, 8102, 2211]",44945.01600694445,только 1 шт. Шиномонтаж на месте -25%,1657251214
2160,96794,NaN,8101,NaN,7365,NaN,Gislaved Urban Speed 185/65R14 86T,Urban Speed,185,65,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Gislaved,https://avatars.mds.yandex.net/get-mpic/156776...,"[8101, 8302]",2023-07-29 18:50:34,NaN,1749412214


In [2]:
print(df.keys())

Index(['PRICE_FRNO', 'PRICE_FNAM', 'PRICE_CAID', 'PRICE_CNAM', 'PRICE_TMNO',
       'Примечание', 'Наименование', 'Модель', 'Ширина шины', 'Высота профиля',
       'Диаметр', 'Индекс нагрузки и скорости', 'XL', 'Unnamed: 13',
       'PRICE_VERT', 'PRICE_PARAM', 'PRICE_PRDS', 'PRICE_DPRN',
       'PRICE_BARCODE', 'PRICE_PTYP', 'PRICE_PROMO', 'PRICE_CURR',
       'PRICE_LOCA', 'PRICE_WARR', 'PRICE_NOTE', 'PRICE_DTSH', 'PRICE_PCOD',
       'date', 'PRICE_RSLT', 'kpl', 'ppl', 'PRICE_IMGN', 'PRICE_LOCD',
       'lineNumber', 'PRICE_SALES', 'PRICE_PLID'],
      dtype='object')


In [3]:
def change_columns(df):   
    new_df = pd.DataFrame()

    df = df.rename(columns={'Наименование': 'offer'})
    df = df[df['offer'].notna()]

    new_df['offer'] = df['offer']
    new_df['width'] = df['Ширина шины']
    new_df['height'] = df['Высота профиля']
    new_df['radius'] = df['Диаметр']
    new_df['v_ind'] = df['Индекс нагрузки и скорости']
    new_df['brand'] = df['ppl']
    new_df['line'] = df['Модель']

    new_df['offer'] = new_df['offer'].apply(process_text)
    new_df['brand'] = new_df['offer'].apply(extract_Gislaved)
    new_df['BIO_Tags'] = new_df.apply(create_bio_tags, axis=1)
    return new_df

def process_text(input_text):
    # Добавляем пробелы к букве "R", если она идет слитно с цифрой
    pattern1 = r'(R)(\d+)'
    processed_text = re.sub(pattern1, r'R \2', input_text)

    pattern2 = r'(\d+)(R)'
    processed_text = re.sub(pattern2, r'\1 R', processed_text)

    pattern3 = r'(\/)(\d+)'
    processed_text = re.sub(pattern3, r'/ \2', processed_text)

    pattern4 = r'(\d+)(\/)'
    processed_text = re.sub(pattern4, r'\1 /', processed_text)

    pattern5 = r'(\\)(\d+)'
    processed_text = re.sub(pattern5, r'\\ \2', processed_text)

    pattern6 = r'(\d+)(\\)'
    processed_text = re.sub(pattern6, r'\1 \\', processed_text)

    # Заменяем символы "/", "\", "|" на пробелы
    # processed_text = re.sub(r'[\/\\|]', ' ', processed_text)

    # Заменяем символы "/" на " / "
    # processed_text = processed_text.replace("/", " / ")
    # Заменяем символы "\" на " \ "
    # processed_text = processed_text.replace("\\", " \\ ")
    # Заменяем символы "|" на " | "
    processed_text = processed_text.replace("|", " | ")
    processed_text = processed_text.replace("(", " ( ")
    processed_text = processed_text.replace(")", " ) ")
    processed_text = processed_text.replace("[", " [ ")
    processed_text = processed_text.replace("]", " ] ")
    # Убираем повторяющиеся пробелы
    processed_text = re.sub(r'  +', ' ', processed_text)
    return processed_text

def create_bio_tags(row):
    name = str(row['offer'])
    width = str(row['width'])
    height = str(row['height'])
    radius = "R" + str(row['radius'])
    brand = row['brand']
    line = str(row['line'])
    v_ind = str(row['v_ind'])

    # Создаем пустой список для хранения тегов
    tags = ['O'] * len(name.split())
    Begin_line = True
    Begin_v_ind = True
    Begin_radius = True
    # Проверяем каждое слово в наименовании и устанавливаем соответствующий тег
    for i, word in enumerate(name.split()):
        if word == '/':
            continue
        # if word == 'R':
        #     continue
        if word == width:
            tags[i] = 'B-width'
        elif word == height:
            tags[i] = 'B-height'
        elif word.upper() in radius.upper():
            if Begin_radius:
                tags[i] = 'B-radius'
                Begin_radius = False
            else:
                tags[i] = 'I-radius'
        elif word in brand:
            tags[i] = 'B-brand'
        elif word.upper() in v_ind.upper():
            if Begin_v_ind:
                tags[i] = 'B-v_ind'
                Begin_v_ind = False
            else:
                tags[i] = 'I-v_ind'  
        elif word.upper() in line.upper() or re.sub(r"[^a-zA-Zа-яА-Я]","", word.upper().replace("СПОРТ", "SPORT")) in line.upper():
            if re.sub(r"[^a-zA-Zа-яА-Я]","", word.upper().replace("СПОРТ", "SPORT")) == "" and Begin_line:
                continue
            if Begin_line:
                tags[i] = 'B-line'
                Begin_line = False
            else:
                tags[i] = 'I-line'
          
                
    # Собираем теги в строку
    return ','.join(tags)

def extract_Gislaved(text):
    brands = re.findall(r'Gislaved|GISLAVED|gislaved|ГИСЛАВЕД|Гиславед|гиславед|Гислейвд|Гислеивд|ГИСЛЕИВД|ГИСЛЕЙВД', text)
    return brands

new_df = change_columns(df)
new_df

,offer,width,height,radius,v_ind,brand,line,BIO_Tags
0,Gislaved Comspeed 185 / 75 R 16 104 / 102 R,185,75,16,104R,[Gislaved],Com Speed,"B-brand,O,B-width,O,B-height,B-radius,I-radius..."
1,Gislaved Comspeed 185 / 75 R 16C 104 / 102 R,185,75,16,104R,[Gislaved],Com Speed,"B-brand,O,B-width,O,B-height,B-radius,B-line,B..."
2,Gislaved Comspeed 185 / 75 R 16C 104 R,185,75,16,104R,[Gislaved],Com Speed,"B-brand,O,B-width,O,B-height,B-radius,B-line,B..."
3,Gislaved COM*SPEED 195 / 60 R 16C 99 / 97T,195,60,16,97T,[Gislaved],Com Speed,"B-brand,O,B-width,O,B-height,B-radius,B-line,I..."
4,185 / 60 R 15 Gislaved Euro Frost 6 88T,185,60,15,88T,[Gislaved],Euro Frost 6,"B-width,O,B-height,B-radius,I-radius,B-brand,B..."
...,...,...,...,...,...,...,...,...
2157,Gislaved Urban Speed 175 / 65 R 13 80T,175,65,13,80T,[Gislaved],Urban Speed,"B-brand,B-line,I-line,B-width,O,B-height,B-rad..."
2158,Gislaved UrbanSpeed 175 / 65 R 13 80T,175,65,13,80T,[Gislaved],Urban Speed,"B-brand,O,B-width,O,B-height,B-radius,I-radius..."
2159,Gislaved UrbanSpeed 185 / 60 R 14 82H,185,60,14,82H,[Gislaved],Urban Speed,"B-brand,O,B-width,O,B-height,B-radius,I-radius..."
2160,Gislaved Urban Speed 185 / 65 R 14 86T,185,65,14,86T,[Gislaved],Urban Speed,"B-brand,B-line,I-line,B-width,O,B-height,B-rad..."


In [4]:
index = 2049
example = dict(new_df.iloc[index])
print(example['offer'],'\n')
offer = example['offer'].split()
BIO_Tags = example['BIO_Tags'].split(',')

for i in range(len(offer)):
    print(f"{offer[i]}: {BIO_Tags[i]}")

Gislaved UltraSpeed 205 / 45 ZR 16 87W 

Gislaved: B-brand
UltraSpeed: B-line
205: B-width
/: O
45: B-height
ZR: O
16: B-radius
87W: B-v_ind


In [5]:
dict(new_df.line.value_counts())

{'Nord Frost 200': 696,
 'Soft Frost 200': 433,
 'Nord Frost 200 SUV': 365,
 'Soft Frost 200 SUV': 201,
 'Nord Frost Van 2': 177,
 'UltraSpeed 2': 109,
 'Nord Frost Van': 62,
 'Euro Frost 6': 55,
 'Nord Frost 100': 11,
 'Soft Frost 200 RunFlat': 9,
 'Nord Frost V': 8,
 'Urban Speed': 6,
 'UltraSpeed': 6,
 'Euro Frost Van': 5,
 'Com Speed': 4,
 'Nord Frost C': 3,
 'Soft Frost 3': 3,
 'Nord Frost 100 SUV': 3,
 'Nord Frost 200 RunFlat': 2,
 'Nord Frost Van RunFlat': 1,
 'Nord Frost III': 1,
 'UltraSpeed 2 SUV': 1,
 'Nord Frost Van  RunFlat': 1}

In [6]:
example

{'offer': 'Gislaved UltraSpeed 205 / 45 ZR 16 87W',
 'width': '205',
 'height': '45',
 'radius': '16',
 'v_ind': '87W',
 'brand': ['Gislaved'],
 'line': 'UltraSpeed 2',
 'BIO_Tags': 'B-brand,B-line,B-width,O,B-height,O,B-radius,B-v_ind'}

In [7]:
example

{'offer': 'Gislaved UltraSpeed 205 / 45 ZR 16 87W',
 'width': '205',
 'height': '45',
 'radius': '16',
 'v_ind': '87W',
 'brand': ['Gislaved'],
 'line': 'UltraSpeed 2',
 'BIO_Tags': 'B-brand,B-line,B-width,O,B-height,O,B-radius,B-v_ind'}

In [8]:
label2id = {'B-width': 1,
            'B-height': 2, 
            'B-radius': 3, 
            'I-radius': 4,
            'B-brand': 5, 
            'B-line': 6, 
            'I-line': 7,
            'B-v_ind': 8,
            'I-v_ind': 9,
            'O': 0}
    
id2label = {1: 'B-width',
            2: 'B-height', 
            3: 'B-radius', 
            4: 'I-radius',
            5: 'B-brand', 
            6: 'B-line', 
            7: 'I-line',
            8: 'B-v_ind',
            9: 'I-v_ind',
            0: 'O'}